![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## AI Inference on a video stream
This notebook is a simple example of how to use DeGirum PySDK to do AI inference on a video stream
in effective pipelined manner using batch predict API.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

You also need to specify your cloud API access token in `degirum_cloud_token`.

You can change `video_source` to index of a local webcamera, or URL of an RTSP stream, or URL of a YouTube video, or path to another video file.


In [ ]:
# make sure degirum-tools package is installed
!pip show degirum || pip install degirum

#### Specify where you want to run your inferences, model zoo url, model name and video source

In [2]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local web camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# degirum_cloud_token: your token for accessing the DeGirum cloud platform
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/example_video.mp4"
degirum_cloud_token = "<enter your token here>"

In [4]:
# Import the necessary libraries
import degirum as dg, cv2

# Connect to AI inference engine
# hw_location: hardware location
# model_zoo_url: cloud model zoo URL
# degirum_cloud_token: Degirum cloud API access token
zoo = dg.connect(hw_location, model_zoo_url, degirum_cloud_token)

# Load the object detection AI model from the model zoo
# model_name: name of the model to load
model = zoo.load_model(
    model_name,
    overlay_show_probabilities=True,  # show probabilities on object labels
    overlay_line_width=1,  # set line width of bounding boxes
)


# Define a frame generator: a function that yields frames from the video stream
def frame_generator(video_source):
    stream = cv2.VideoCapture(video_source)  # open the video stream
    try:
        while True:
            ret, frame = stream.read()  # read a frame from the video stream
            if not ret:  # if the frame was not read successfully, break the loop
                break
            yield frame  # yield the frame
    finally:
        stream.release()  # finally release the video capture object to free up resources


# Process the video stream by AI model using model.predict_batch():
# an efficient method for pipelined processing of video streams.
# The result is an object that includes the processed frame and other information
for result in model.predict_batch(frame_generator(video_source)):
    # Display the frame with AI annotations in a window named 'AI Inference'
    cv2.imshow("AI Inference", result.image_overlay)

    # Process GUI events and break the loop if 'q' key was pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()  # destroy any remaining OpenCV windows